In [1]:
# Integer class
#
class BigInt:
    def __init__(self, a=0):
        assert(isinstance(a, int))
        if a >= 0:
            self.sign = 1
        else:
            self.sign = -1
            a = -a
        self.digits = []
        while a > 0:                   #save the digits in reverse order as a list
            self.digits.append(a % 10)
            a = a // 10
        if len(self.digits) == 0:
            self.digits.append(0)
    
    
    def __str__(self):
        result = [str(d) for d in self.digits]
        result.append('' if self.sign > 0 else '-')
        reverse = result[::-1]
        return ''.join(reverse)

    
    def FlipSign(self):
        self.sign = -1 * self.sign
        
    
    def Normalize(self):
        k = len(self.digits) - 1
        while k >= 0:
            if self.digits[k] != 0: #ignore the zeros at the end of the list
                return
            self.digits.pop()
            k = k-1
        if len(self.digits) == 0:
            self.digits.append(0)
            self.sign = 1
        if self.digits == [0]:
            self.sign = 1

In [2]:
ia = BigInt(-123)
print(ia.digits)
ia.FlipSign()
print(ia)

ia.digits = [1, 2, 0, 3, 0, 0, 0]
print(ia.__str__())
ia.Normalize()
print(ia)
ia.digits = [7, 0]
print(ia)
ia.Normalize()
print(ia)

[3, 2, 1]
123
0003021
3021
07
7


In [3]:
# Operation of integers
#
def BigIntAbs(a):
    b = BigInt()
    b.digits = a.digits
    b.sign = 1
    return b


# Assume a, b have no leading zeros.
# Returns 1 if a > b, -1 if a < b, 0 if a == b
def _CompareDigits(a, b):
    if len(a) > len(b):
        return 1
    if len(a) < len(b):
        return -1
    k = len(a) - 1
    while k >= 0:
        if a[k] > b[k]:
            return 1
        elif a[k] < b[k]:
            return -1
        k = k - 1
    return 0


# Assume a and b are normalized.
# Returns 1 if a > b, -1 if a < b, 0 if a==b
def CompareBigInts(a, b):
    if a.sign > 0 and b.sign < 0:
        return 1
    if a.sign < 0 and b.sign > 0:
        return -1
    return _CompareDigits(a.digits, b.digits) * (1 if a.sign > 0 else -1)


# Assume a and b are normalized.
def BigIntEquals(a, b):
    return CompareBigInts(a, b) == 0

In [4]:
ia = BigInt(-123)
print(BigIntAbs(ia))
ib = BigInt(123)
print(BigIntEquals(ia, ib))
ib.FlipSign()
print(BigIntEquals(ia, ib))
ic = BigInt(3)
id = BigInt(4)
print(CompareBigInts(ic, id))
print(CompareBigInts(id, ic))
print(CompareBigInts(ia, ib))

123
False
True
-1
1
0


In [5]:
def _AddDigits(a, b):
    c = []
    k = 0
    carry_on = 0
    while k < len(a) or k < len(b):
        s = carry_on
        if k < len(a):
            s = s + a[k]
        if k < len(b):
            s = s + b[k]
        if s >= 10:
            carry_on = 1
            c.append(s - 10)
        else:
            carry_on = 0
            c.append(s)
        k = k + 1
    if carry_on != 0:    
        c.append(carry_on)  
    return c


# Assume that a > b >= 0, len(a)>=len(b)
def _SubtractDigits(a, b):
    c = []
    k = 0
    borrow = 0
    while k < len(a):
        if k < len(b):
            if b[k] <= a[k] - borrow:
                c.append(a[k] - borrow - b[k])
                borrow = 0
            if b[k] > a[k] - borrow:
                c.append(10 + a[k] - borrow - b[k])
                borrow = 1
        if k >= len(b) and k < len(a) - 1:
            if a[k] - borrow >= 0:
                c.append(a[k] - borrow)
                borrow = 0
            if a[k] - borrow < 0:
                c.append(10 + a[k] - borrow)
                borrow = 1
        if k >= len(b) and k == len(a) - 1:
            if a[k] - borrow > 0:
                c.append(a[k] - borrow)      
        k += 1
    return c


def BigIntAdd(a, b):
    if a.sign == b.sign:
        c = BigInt(0)
        c.sign = a.sign
        c.digits = _AddDigits(a.digits, b.digits)
        return c
    cmp = _CompareDigits(a.digits, b.digits)
    if cmp == 0:
        return BigInt(0)
    elif cmp > 0:
        c = BigInt(0)
        c.sign = a.sign
        c.digits = _SubtractDigits(a.digits, b.digits)
        return c
    else:
        c = BigInt(0)
        c.sign = b.sign
        c.digits = _SubtractDigits(b.digits, a.digits)
        return c


def BigIntSubtract(a, b):
    if a.sign != b.sign:
        c = BigInt(0)
        c.sign = a.sign
        c.digits = _AddDigits(a.digits, b.digits)
        return c
    cmp = _CompareDigits(a.digits, b.digits)
    if cmp == 0:
        return BigInt(0)
    elif cmp > 0:
        c = BigInt(0)
        c.sign = a.sign
        c.digits = _SubtractDigits(a.digits, b.digits)
        c.Normalize()
        return c
    else:
        c = BigInt(0)
        c.sign = -1 *  a.sign
        c.digits = _SubtractDigits(b.digits, a.digits)
        c.Normalize()
        return c



In [6]:
ia = BigInt(9)
ib = BigInt(1)
print(ia.digits)
print(ib.digits)
print(BigIntAdd(ia, ib))
print(BigIntSubtract(ia, ib))
print(_AddDigits(ia.digits, ib.digits))

[9]
[1]
10
8
[0, 1]


In [7]:
# * 
def _MultiplyWithSingleDigit(a, b):
    c = []
    assert(isinstance(b, int) and 0 <= b <= 9)
    k = 0
    carry_on = 0
    while k < len(a):
        s = a[k] * b + carry_on
        c.append(s % 10)
        carry_on = s // 10
        k = k + 1
    c.append(carry_on)    
    return c

def _MultiplyWithPowersOfTen(a, k):
    c = []
    if k == 0:
        return a
    for i in range(k):
        c.append(0)
    i = 0    
    while i < len(a):
        c.append(a[i])
        i += 1
    return c

def _MultiplyDigits(a, b):
    c = [0]
    for k in range(len(b)):
        s = _MultiplyWithSingleDigit(a, b[k])
        s = _MultiplyWithPowersOfTen(s, k)
        c = _AddDigits(c, s)
    return c

def BigIntMultiply(a, b):
    c = BigInt(0)
    c.sign = a.sign * b.sign
    c.digits = _MultiplyDigits(a.digits, b.digits)
    c.Normalize()
    return c

def BigIntPower(a, n):
    assert(n >= 0)
    c = BigInt(1)
    
    for i in range(n):
        c = BigIntMultiply(c, a)
    return c

def BigIntSlowerDivide(a, b):
    assert(b != BigInt(0))
    q = BigInt(0)
    r = BigInt(0)
    q.digits = [0]
    r.digits = a.digits
    while CompareBigInts(r, b) >= 0:
        q.digits = _AddDigits(q.digits, [1])
        r.digits = _SubtractDigits(r.digits, b.digits)  
    q.sign = a.sign * b.sign
    r.sign = a.sign
    r.Normalize()
    q.Normalize()
    return q, r


def BigIntDivide(a, b):
    assert(b != BigInt(0))
    q = BigInt(0)
    # if the size of a is less than b, simply return a as the remainder
    if CompareBigInts(BigIntAbs(a), BigIntAbs(b)) < 0: 
        return q, a
    
    r = BigInt(0)
    lb = len(b.digits)
    la = len(a.digits)
    r.digits = a.digits[la-lb:la] # match the number of digits
    
    i = 0
    while i < la - lb: 
        s = 0
        while CompareBigInts(r, b) >= 0: # find the integer part of quotient
            r = BigIntSubtract(r, b)
            r.Normalize()
            s += 1
        q.digits.append(s)
        
        ll = r.digits[::-1]         # attach the next digit of a to the remainder
        ll.append(a.digits[la-lb-1-i])
        r.digits = ll[::-1]
        i += 1
    
    s = 0
    r.Normalize()  # normalize r before comparing with b
    while CompareBigInts(r, b) >= 0: # find the last digit of q
        r = BigIntSubtract(r, b)
        r.Normalize()
        s += 1
    q.digits.append(s)
      
    q.digits = q.digits[::-1]
    r.Normalize()
    q.Normalize()
    return q, r

In [8]:
a = BigInt(40)
b = BigInt(113)

q, r = BigIntDivide(a, b)
#q, r = BigIntQuickerDivide(a, b)
print(q)
print(r)


0
40


In [9]:
# Fraction class
# BigFraction add, subtract, multiply, divide, power
class BigFraction:
    def __init__(this, numerator=BigInt(0), denominator=BigInt(1)):
        this.numerator = numerator
        this.denominator = denominator
        if this.denominator.sign < 0:
            this.denominator.FlipSign()
            this.numerator.FlipSign()
            
    
    
    def __str__(this):
        return str(this.numerator) + '/' + str(this.denominator)

    
def BigFractionAdd(a, b):
    c = BigFraction(BigInt(1), BigInt(1))
    c.denominator = BigIntMultiply(a.denominator, b.denominator)
    c.numerator = BigIntAdd(BigIntMultiply(a.denominator, b.numerator), BigIntMultiply(a.numerator, b.denominator))
    return c

def BigFractionSubtract(a, b):
    c = BigFraction(BigInt(1), BigInt(1))
    c.denominator = BigIntMultiply(a.denominator, b.denominator)
    c.numerator = BigIntSubtract(BigIntMultiply(a.numerator, b.denominator), BigIntMultiply(a.denominator, b.numerator))
    return c

def BigFractionMultiply(a, b):
    c = BigFraction(BigInt(1), BigInt(1))
    c.denominator = BigIntMultiply(a.denominator, b.denominator)
    c.numerator = BigIntMultiply(a.numerator, b.numerator)
    return c

def BigFractionPower(a, n):
    assert(n >= 0)
    c = BigFraction(BigInt(1), BigInt(1))
    
    for i in range(n):
        c = BigFractionMultiply(c, a)
    return c

In [10]:
fa = BigFraction(BigInt(2), BigInt(-3))
print(fa)
fb = BigFraction()
fc = BigFraction(BigInt(5), BigInt(4))
print(BigFractionSubtract(fc, fa))
print(BigFractionPower(fa, 3))
print(BigIntMultiply(BigInt(2), BigInt(10)))

-2/3
23/12
-8/27
20


In [11]:
def BigFractionToDecimal(fa, k): #represent a Big Fraction up to k decimal places
    nu = fa.numerator 
    de = fa.denominator
    l = []
    q, r = BigIntDivide(nu, de)
    l.append(q)
    l.append('.')
    i = 0
    for i in range(k):
        q, r = BigIntDivide(BigIntMultiply(r, BigInt(10)), de)
        l.append(q)
    l = [str(d) for d in l]    
    return ''.join(l)

def BigFractionToDecimal1(fa, k): #represent a Big Fraction up to k decimal places using the slower divide method
    nu = fa.numerator 
    de = fa.denominator
    l = []
    q, r = BigIntDivide(nu, de)
    l.append(q)
    l.append('.')
    i = 0
    for i in range(k):
        q, r = BigIntSlowerDivide(BigIntMultiply(r, BigInt(10)), de)
        l.append(q)
    l = [str(d) for d in l]    
    return ''.join(l)

In [12]:
fa = BigFraction(BigInt(355), BigInt(113))
print(BigFractionToDecimal(fa, 11))
print(BigFractionToDecimal1(fa, 11))

3.14159292035
3.14159292035


In [13]:
#Taylor polynomial of arctan up to n-th term 
def TaylorPolyOfArctan(x, n): # x should be a big fraction
    s = BigFraction()
    for i in range(n+1):
        p = 2*i + 1
        next_term = BigFractionMultiply(BigFractionPower(x, p), BigFraction(BigInt((-1)**i), BigInt(p)))
        s  = BigFractionAdd(s, next_term) 
    return s    

In [14]:
#use Machin's formula to estimate pi
x = BigFraction(BigInt(1), BigInt(5))
y = BigFraction(BigInt(1), BigInt(239))

#use Taylor polybomial up to n-th term
n = 30
s1 = BigFractionMultiply(TaylorPolyOfArctan(x, n), BigFraction(BigInt(4), BigInt(1)))
s2 = TaylorPolyOfArctan(y, n)

#s2 = BigFractionMultiply(s2, BigFraction(BigInt(-1), BigInt(1)))

s = BigFractionSubtract(s1, s2)
result = BigFractionMultiply(s, BigFraction(BigInt(4), BigInt(1)))

#represent result as a decimal
result = BigFractionToDecimal(result, 100)
print(result)


3.1415926535897932384626433832795028841971694016301271684489234984290065900959680994986868403485258598


In [17]:
# 3.
# 1415926535 8979323846 2643383279 5028841971 6939937510 
# 5820974944 5923078164 0628620899 8628034825 3421170679 